In [1]:
# Install required packages in the current Jupyter kernel
# Uncomment the following lines if you need to install these libraries
# If you run into permission issues, try with the --user option
#import sys
#!pip install -q rdflib networkx matplotlib
#!{sys.executable} -m pip install rdflib networkx matplotlib pandas stringdb --user


In [15]:
%load_ext autoreload
%autoreload 2
import sampler
import grabhogs_sparql
import map2string_fast
import addfrombloom
import rdflib
import SPARQLWrapper
import colour
import itertools
import networkx as nx
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
datapath = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.links.rdf.v11.5/402676.protein.links.rdf.v11.5.txt.gz'
# RDF graph loading
rg = sampler.load_graph(datapath)

datapath2 = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/datasets/STRING/rdf/protein.info.rdf.v11.5/402676.protein.info.rdf.v11.5.txt.gz'
# RDF graph loading
rg_info  = sampler.load_graph(datapath2)

rdflib Graph loaded successfully with 2620310 triples
rdflib Graph loaded successfully with 14559 triples


In [17]:
subjs = rg.subjects( unique = True)
seed = next(subjs)
print(seed)

https://string-db.org/network/402676.B6K2P4


In [46]:
subg = sampler.sample( rg = rg , seed = seed,  layer_limit= 2 , sample_run = 20 )
print(set([p for p in subg.predicates()]))
print("rdflib Graph sampled successfully with {} triples".format(len(subg)))

176 704
{rdflib.term.URIRef('https://string-db.org/rdf/high-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/medium-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/any-confidence'), rdflib.term.URIRef('https://string-db.org/rdf/highest-confidence-cutoff')}
rdflib Graph sampled successfully with 560 triples


In [47]:
subg = sampler.add_xrefs( rg_info , subg )
print("rdflib Graph annotated successfully with {} triples".format(len(subg)))
print(set([p for p in subg.predicates()]))
cross_ref = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot")
i = 0 
for s,p,o in subg.triples((None, cross_ref, None)):
    print(s,p,o)
    i+= 1
    if i > 10:
        break

51
51
rdflib Graph annotated successfully with 611 triples
{rdflib.term.URIRef('https://string-db.org/rdf/any-confidence'), rdflib.term.URIRef('https://string-db.org/rdf/highest-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/medium-confidence-cutoff'), rdflib.term.URIRef('https://string-db.org/rdf/high-confidence-cutoff'), rdflib.term.URIRef('http://purl.org/lscr#xrefUniprot')}
https://string-db.org/network/402676.B6K3E3 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6K3E3
https://string-db.org/network/402676.B6K1B5 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6K1B5
https://string-db.org/network/402676.B6JVX3 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JVX3
https://string-db.org/network/402676.B6JWI9 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JWI9
https://string-db.org/network/402676.B6JVR3 http://purl.org/lscr#xrefUniprot http://purl.uniprot.org/uniprot/B6JVR3
https://string-db.o

In [48]:
proteins_by_species , results_subj_para , results_subj_ortho  = grabhogs_sparql.grab_hogs( subg , cross_ref = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot") )

51
Total time for 102 SPARQL queries: 15.509922981262207 seconds (multiple batch calls in: 0 cases)
Num errors: 0


In [49]:
print(results_subj_ortho)

                                        protein1  \
0     https://omabrowser.org/oma/info/SCHJY01298   
1     https://omabrowser.org/oma/info/SCHJY01298   
2     https://omabrowser.org/oma/info/SCHJY01298   
3     https://omabrowser.org/oma/info/SCHJY01298   
4     https://omabrowser.org/oma/info/SCHJY01298   
...                                          ...   
3802  https://omabrowser.org/oma/info/SCHJY01298   
3803  https://omabrowser.org/oma/info/SCHJY01298   
3804  https://omabrowser.org/oma/info/SCHJY01298   
3805  https://omabrowser.org/oma/info/SCHJY01298   
3806  https://omabrowser.org/oma/info/SCHJY01298   

                            protein1_uniprot  \
0     http://purl.uniprot.org/uniprot/B6JW05   
1     http://purl.uniprot.org/uniprot/B6JW05   
2     http://purl.uniprot.org/uniprot/B6JW05   
3     http://purl.uniprot.org/uniprot/B6JW05   
4     http://purl.uniprot.org/uniprot/B6JW05   
...                                      ...   
3802  http://purl.uniprot.org/uniprot/B

In [50]:
orthograph =  grabhogs_sparql.grab_hogs_graph( subg , cross_ref , sparql_endpoint= None
            , USE_CASE = 1 , verbose = True , cross_ref_prop = rdflib.term.URIRef("http://purl.org/lscr#xrefUniprot"))

51
Total time for 102 SPARQL queries: 37.98742079734802 seconds (multiple batch calls in: 0 cases)
Num errors: 0


In [51]:
from rdflib import Graph, URIRef
print("rdflib Graph annotated successfully with {} triples".format(len(orthograph)))
#get all species
taxa = [ 'protein1_uniprot_taxon_orth' , 'protein1_uniprot_taxon_para']
species = set( [ o for t in taxa  for s,p,o in orthograph.triples((None, URIRef(t), None))  ])


rdflib Graph annotated successfully with 576061 triples


In [52]:
#get all the results for that species
taxa = [ 'orth_protein_uniprot_taxon_orth' , 'para_protein_uniprot_taxon_para']
#get all proteins for each species
prots_by_species = { spec: set([s  for s,p,o in orthograph.triples((None, URIRef(t), spec ))]) for t in taxa for spec in species  }
prots_by_species = { spec:prots_by_species[spec] for spec in prots_by_species if len(prots_by_species[spec])   }

In [53]:
server = 'dna079'
ortho_xrefgraph = map2string_fast.mapall(proteins_by_species , serverurl= "http://"+server+":3030/string_fuseki/sparql" , retgraph = True)

In [54]:
orthograph += ortho_xrefgraph
subg += orthograph
#we have interactions for one species and ortho info to all others

In [56]:
filters = addfrombloom.load_filters()

In [63]:
#lets add the interactions for all using the bloom filters

print( len(orthograph ) , set([p for p in orthograph.predicates() ]) )
#get string ids by species
pred = rdflib.term.URIRef('http://purl.org/lscr#xrefUniprot')
interactions = []
for spec in prots_by_species:
    stringids = [ s for prot in prots_by_species[spec] for s,p,o in orthograph.triples((None, pred , prot )) ]
    stringids = [ s.replace('https://string-db.org/network/' , '' ) for s in stringids ]
    if len(stringids ) > 2 :
        interactions += addfrombloom.check_allvall( objects = stringids , urlstring = 'https://string-db.org/network/' , filters = filters )
[subg.add(t) for t in interactions]

606219 {rdflib.term.URIRef('protein1_para_protein_uniprot'), rdflib.term.URIRef('para_protein_para_protein_uniprot'), rdflib.term.URIRef('orth_protein_orth_protein_uniprot'), rdflib.term.URIRef('protein1_orth_protein'), rdflib.term.URIRef('protein1_taxon'), rdflib.term.URIRef('taxon_taxon_orth'), rdflib.term.URIRef('taxon_orth_protein_uniprot'), rdflib.term.URIRef('taxon_para_protein_uniprot'), rdflib.term.URIRef('para_protein_uniprot_taxon_para'), rdflib.term.URIRef('protein1_uniprot_para_protein'), rdflib.term.URIRef('protein1_uniprot_para_protein_uniprot'), rdflib.term.URIRef('protein1_para_protein'), rdflib.term.URIRef('taxon_para_protein'), rdflib.term.URIRef('protein1_uniprot_orth_protein'), rdflib.term.URIRef('protein1_taxon_orth'), rdflib.term.URIRef('protein1_uniprot_taxon'), rdflib.term.URIRef('protein1_uniprot_taxon_para'), rdflib.term.URIRef('taxon_orth_protein'), rdflib.term.URIRef('orth_protein_uniprot_taxon_orth'), rdflib.term.URIRef('protein1_taxon_para'), rdflib.term.U

[<Graph identifier=N22f688f2ecc5470fa62db0c655d0b3f4 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N22f688f2ecc5470fa62db0c655d0b3f4 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N22f688f2ecc5470fa62db0c655d0b3f4 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N22f688f2ecc5470fa62db0c655d0b3f4 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N22f688f2ecc5470fa62db0c655d0b3f4 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N22f688f2ecc5470fa62db0c655d0b3f4 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N22f688f2ecc5470fa62db0c655d0b3f4 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N22f688f2ecc5470fa62db0c655d0b3f4 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N22f688f2ecc5470fa62db0c655d0b3f4 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N22f688f2ecc5470fa62db0c655d0b3f4 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N22f688f2ecc5470fa62db0c655d0b3f4 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N22f688f2ecc5470fa62db0c655d0b3f4 (

In [ ]:
#halelujah we have a graph with everything in it


In [ ]:
G = rdflib.extras.external_graph_libs.rdflib_to_networkx_multidigraph( subg , lambda s, p, o: {'data':{'key':p  , 'weight':1} } )

In [ ]:
#sample response
import time
start_time = time.time()
results_obj_para = grabhogs_sparql.query_orthologs("<http://purl.uniprot.org/uniprot/B6JXK8>", SPARQLWrapper.SPARQLWrapper("https://sparql.omabrowser.org/sparql/"))
end_time = time.time()
print("Total time : {} seconds".format(end_time - start_time))

In [ ]:
# example usage
uniprot_id = "http://purl.uniprot.org/uniprot/A0A3B5R6M3" 
serverurl = '
results =  map2string_fast.map2string_SPARQL(uniprot_id , serverurl = )
print(results)

In [ ]:
preds = [ p for p in rg.predicates( unique = True)]
red = colour.Color('red')
blue = colour.Color('blue')
c = [ c.hex_l for c in  list(red.range_to(blue, len(preds))) ]
colors = { p:c[i] for i,p in enumerate(preds)}
delta = 2/len(preds)
curve = { p:delta*i for i,p in enumerate(preds) }
style = itertools.cycle([ '-', '--' ])
line_style = { p:next(style) for p in preds }

In [ ]:
pos = nx.circular_layout( G )
f = plt.figure()
f.set_figwidth(15)
f.set_figheight(15)
plt.plot()
#plot the whole mess
ax = plt.gca()
for e in G.edges(data = True):
    ax.annotate("",
                xy=pos[e[0]], xycoords='data',
                xytext=pos[e[1]], textcoords='data',
                arrowprops=dict(arrowstyle="-", color=colors[e[2]['data']['key']],
                                shrinkA=5, shrinkB=5, lw = e[2]['data']['weight'], ls = line_style[e[2]['data']['key']],
                                patchA=None, patchB=None, alpha = e[2]['data']['weight'],
                                connectionstyle="arc3,rad=rrr".replace('rrr',str(0.3*curve[e[2]['data']['key']])
                                ),
                                ),
                )

nx.draw_networkx_nodes(G, pos, node_color = 'b', node_size = 150, alpha = .5)
labels=nx.draw_networkx_labels(G , pos = pos , font_size= 15 , font_color='w')

plt.axis('off')
plt.show()


In [ ]:
#load bloom filters for string data
import pickle
import glob
def check_filters(val,filters = filters):
    for f in filters:
        if val in f[0][0]:
            return True
    return False
filters = './filters/bloomfinal_big*.pkl'
filters = glob.glob(filters)
filters = [ pickle.load(open(f, 'rb')) for f in filters ]
#check bloom filters for interactors in diff species
print(check_filters('crap'))
print(check_filters('23.BEL05_00185 _23.BEL05_00025 '.encode()))
print(check_filters('23.BEL05_00045 23.BEL05_02665'))

In [ ]:
#map string neighbours to OMA entries

#jump a few steps in HOGs

#fish for STRING interactions in other species


